In [3]:
!pip install openai langchain-community langchain-openai langgraph crewai-tools langchain-chroma langchain langchain-text-splitters langchain_core langchain_huggingface pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 8.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of crewai to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-chroma to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-http to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.9 MB/s eta 0:00:

In [4]:
!pip install langchain-community chromadb langchain-text-splitters sentence-transformers


In [5]:

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_community.embeddings import SentenceTransformerEmbeddings

from langchain_core.prompts import PromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain


In [6]:
BASE_URL = "https://openrouter.ai/api/v1"
OPENROUTER_API_KEY = "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"
MODEL_1 = "mistralai/devstral-2512:free"
EMBEDDING_MODEL='all-MiniLM-L6-v2'
PDF_PATH="/content/Umair_Ali.pdf"
PERSIST_DIRECTORY='/content/chromadb_rag'
CHUNK_SIZE=200
CHUNK_OVERLAP=50

prompt_template='''
Your are a powerful AI assistant

Your task is to provide valid information of applier after reading the resume.
read retrieved information from database carefully read it and generate appropiate answer

retrieved_chunks:{text}
user_query:{query}
'''

In [7]:
llm = ChatOpenAI(
    model=MODEL_1,
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base=BASE_URL,
    temperature=0.7,
    max_tokens=1024,
    default_headers={
        "HTTP-Referer": "https://company-chatbot.local",
        "X-Title": "Multi-Agent System",
    }
)

In [8]:
def document_loader(pdf_path):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    return docs


In [9]:
def text_splitter(docs):
  splitter=RecursiveCharacterTextSplitter(
      chunk_size=CHUNK_SIZE,
      chunk_overlap=CHUNK_OVERLAP,
      separators=['\n\n','\n',' ']
  )

  chunks=splitter.split_documents(docs)
  return  chunks

In [10]:
def build_chroma_db(
    documents,
    persist_directory="/content/chroma_db",
    model_name="all-MiniLM-L6-v2"
):

    embedding = SentenceTransformerEmbeddings(
        model_name=model_name
    )

    vectordb = Chroma.from_documents(
        documents=documents,
        embedding=embedding,
        persist_directory=persist_directory
    )

    vectordb.persist()
    return vectordb


In [11]:
def load_chroma_db(
    persist_directory="/content/chroma_db",
    model_name="all-MiniLM-L6-v2"
):
    embedding = SentenceTransformerEmbeddings(
        model_name=model_name
    )

    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )

    return vectordb


In [12]:
def similarity_search(
    vectordb,
    query,
    k=3
):
    results = vectordb.similarity_search(
        query=query,
        k=k
    )
    return results


In [20]:


def chat_bot(query, vector_db, top_k=5):

    retrieval = similarity_search(vector_db, query, k=top_k)


    context_text = " ".join([doc.page_content for doc in retrieval])
    prompt = PromptTemplate(
        input_variables=["text", "query"],
        template=prompt_template
    )

    final_prompt = prompt.format(text=context_text, query=query)
    response = llm.invoke(final_prompt)

    return response


In [14]:
# test llm model
# print(llm.invoke("Hi"))



# test Document Loading and Chunking
docs=document_loader(PDF_PATH)
chunks=text_splitter(docs)
chunks_list=[chunk.page_content for chunk in chunks]
# for i, chunk in enumerate(chunks_list):
#   print(i,"......", chunk,"\n")


build_chroma_db(chunks)

/tmp/ipython-input-1841052864.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-1841052864.py:17: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [22]:
vector_db = load_chroma_db()
query = "what is education and work experience and education? "
# retrieval = similarity_search(vector_db, query, k=5)
# retrieved_text=" ".join([ret.page_content for ret in retrieval])
# print(retrieved_text)

print(chat_bot(query,vector_db).content)

Here’s a structured summary of the applicant’s **education** and **work experience** based on the retrieved information:

### **Education**
1. **MS in Data Science**
   - **Institution**: Information Technology University (ITU)
   - **Duration**: 2024–Present

2. **Certifications & Courses**
   - **Deep Learning Specialization** (Coursera, Andrew Ng)
   - **Machine Learning with Python** (IBM Skills Network)
   - **Customer QA ChatBot** (Python + RAG, OpenAI, LangChain, FastAPI, Docker, AWS, Pinecone)
   - **Digital Economic Expert** (LangChain, Claude, RAG, Python, FAISS)

### **Work Experience**
1. **Associate Data Scientist**
   - **Company**: ML1.Pvt
   - **Location**: Lahore, Pakistan
   - **Duration**: Jun 2025 – Nov 2025
   - **Responsibilities**:
     - Data preparation
     - Pipeline and model building

2. **Research Assistant**
   - **Institution**: Information Technology University (ITU)
   - **Location**: Lahore, Pakistan
   - **Duration**: 2024–2025

### **Key Skills & To